In [1]:
import os

import data_loader
import model
import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import percentage_split

W0313 06:38:28.503304 4665269696 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
import warnings
warnings.filterwarnings('ignore')
tf.logging.set_verbosity(tf.logging.ERROR)

In [16]:
training_data_paths = [
        '../data-capsnets/scenario0/train.txt',
        '../data-capsnets/scenario1/train.txt',
        '../data-capsnets/scenario2/train.txt',
        '../data-capsnets/scenario3.1/train.txt',
        '../data-capsnets/scenario3.2/train.txt',
        '../data-capsnets/scenario3.3/train.txt',
    ]
test_data_paths = [
    '../data-capsnets/scenario0/test.txt',
    '../data-capsnets/scenario1/test.txt',
    '../data-capsnets/scenario2/test.txt',
    '../data-capsnets/scenario3.1/test.txt',
    '../data-capsnets/scenario3.2/test.txt',
    '../data-capsnets/scenario3.3/test.txt',
]
scenario_nums = [
    '0_perc',
    '1_perc',
    '2_perc',
    '31_perc',
    '32_perc',
    '33_perc',
]


In [17]:
run_results = dict()

In [5]:
def load_w2v(file_name):
    """ Load Word2Vec model
        Args:
            file_name: model file name
        Returns:
            w2v: w2v model
    """
    w2v = KeyedVectors.load_word2vec_format(
        file_name, binary=False)
    return w2v

In [6]:
from gensim.models.keyedvectors import KeyedVectors

# load word2vec model
word2vec_path = '../../romanian_word_vecs/cc.ro.300.vec'

print('------------------load word2vec begin-------------------')
w2v = load_w2v(word2vec_path)
print('------------------load word2vec end---------------------')

------------------load word2vec begin-------------------
------------------load word2vec end---------------------


In [7]:
import util 

def read_datasets(w2v, training_data_path, test_data_path, test=False):
    # TODO: split read into read train and read test
    """ Encodes the intent and slot labels in one-hot format
        Args:
            test: True if running test on the model -- this will load the test data in raw text format
        Returns:
            data: data dictionary\
    """
    print('------------------read datasets begin-------------------')
    data = {}

   

    # load normalized word embeddings
    embedding = w2v.vectors
    norm_embedding = util.norm_matrix(embedding)
    data['embedding'] = norm_embedding

    # trans data into embedding vectors
    max_len = 0
    slots_dict = dict()
    intents_dict = dict()
    slot_id = 0
    intent_id = 0
    (x_tr, y_intents_tr, y_slots_tr, sentences_length_tr,
     max_len, intents_dict, intent_id, slots_dict, slot_id,
     x_text_tr) = data_loader.load_vec(training_data_path, w2v, max_len,
                           intents_dict, intent_id, slots_dict, slot_id,
                           load_text=False)
    (x_te, y_intents_te, y_slots_te, sentences_length_te,
     max_len, intents_dict, intent_id, slots_dict, slot_id,
     x_text_te) = data_loader.load_vec(test_data_path, w2v, max_len, intents_dict,
                           intent_id, slots_dict, slot_id,
                           load_text=test)
    intents_id_dict = {v: k for k, v in intents_dict.items()}
    slots_id_dict = {v: k for k, v in slots_dict.items()}

    data['x_tr'] = x_tr
    data['y_intents_tr'] = y_intents_tr
    data['y_slots_tr'] = y_slots_tr
    data['sentences_len_tr'] = sentences_length_tr

    data['intents_dict'] = intents_id_dict
    data['slots_dict'] = slots_id_dict

    data['x_te'] = x_te
    data['y_intents_te'] = y_intents_te
    data['y_slots_te'] = y_slots_te
    data['sentences_len_te'] = sentences_length_te

    if test:
        data['x_text_te'] = x_text_te

    data['max_len'] = max_len

    one_hot_y_intents_tr, one_hot_y_slots_tr = data_loader.get_label(data)
    data['encoded_intents_tr'] = one_hot_y_intents_tr
    data['encoded_slots_tr'] = one_hot_y_slots_tr

    one_hot_y_intents_te, one_hot_y_slots_te = data_loader.get_label(data, test=True)
    data['encoded_intents_te'] = one_hot_y_intents_te
    data['encoded_slots_te'] = one_hot_y_slots_te
    print('------------------read datasets end---------------------')
    return data


In [17]:
%debug

In [19]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_boolean('save_model', False, 'save model to disk')
tf.app.flags.DEFINE_string('summaries_dir', './logs', 'tensorboard summaries')
tf.app.flags.DEFINE_string('ckpt_dir', './saved_models/', 'check point dir')
tf.app.flags.DEFINE_string('errors_dir', './errors/', 'Errors dir')
        
hidden_size = 64

tf.app.flags.DEFINE_float('keep_prob', 0.8, 'embedding dropout keep rate for training')
tf.app.flags.DEFINE_integer('hidden_size', hidden_size, 'embedding vector size')
tf.app.flags.DEFINE_integer('batch_size', 32, 'batch size')
tf.app.flags.DEFINE_integer('num_epochs', 20, 'num of epochs')
tf.app.flags.DEFINE_boolean('use_embedding', True, 'whether to use embedding or not.')
tf.app.flags.DEFINE_float('learning_rate', 0.01, 'learning rate')
tf.app.flags.DEFINE_float('margin', 1.0, 'ranking loss margin')
tf.app.flags.DEFINE_integer('slot_routing_num', 2, 'slot routing num')
tf.app.flags.DEFINE_integer('intent_routing_num', 3, 'intent routing num')
tf.app.flags.DEFINE_integer('intent_output_dim', 16, 'intent output dimension')
tf.app.flags.DEFINE_integer('slot_output_dim', 2 * hidden_size, 'slot output dimension')
tf.app.flags.DEFINE_integer('d_a', 20, 'self attention weight hidden units number')
tf.app.flags.DEFINE_integer('r', 5, 'number of self attention heads')
tf.app.flags.DEFINE_float('alpha', 0.0001, 'coefficient for self attention loss')
tf.app.flags.DEFINE_integer('n_splits', 3, 'Number of cross-validation splits')
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [18]:
import flags
flags.del_all_flags(FLAGS)

In [9]:
def del_some_flags(fl):
    flags_dict = fl._flags()
    keys_list = [keys for keys in flags_dict]
    to_delete = ['scenario_num', 'vocab_size', 'max_sentence_length', 'intents_nr', 'slots_nr', 'word_emb_size']
    for keys in keys_list:
        if keys in to_delete:
            fl.__delattr__(keys)

In [13]:
a = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
b = ['a','a','b','b','c','c','d','d','e','e','a','a','b','b','c','c','d','d','e','e']
for t_i, v_i in StratifiedShuffleSplit(n_splits=2, train_size=0.5).split(a,b):
    print(t_i, v_i)
    print(len(t_i))
    tivi = np.concatenate((t_i, v_i))
    tivi = np.sort(tivi)
    print(tivi)

[ 9  1  2  5 11 14  7  3  6 19] [18 15  0 12 17 13 16 10  4  8]
10
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 8 18 14  0 16 10  2 12 17 15] [ 3  4 19 13  9  7 11  5  6  1]
10
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [20]:
for i in range(len(training_data_paths)):
    # Define the flags
#     FLAGS = flags.define_app_flags(scenario_num=scenario_nums[i])

    # Load data
    print('Reading training data from ' + training_data_paths[i])
    print('Reading test data from ' + training_data_paths[i])
    print('Scenario num ' + scenario_nums[i])
    data = read_datasets(w2v, training_data_paths[i], test_data_paths[i], test=True)
    x_tr = data['x_tr']
    y_intents_tr = data['y_intents_tr']
    y_slots_tr = data['y_slots_tr']
    sentences_length_tr = data['sentences_len_tr']

    one_hot_intents_tr = data['encoded_intents_tr']
    one_hot_slots_tr = data['encoded_slots_tr']

    x_te = data['x_te']
    x_text_te = data['x_text_te']
    y_intents_te = data['y_intents_te']
    y_slots_te = data['y_slots_te']
    sentences_length_te = data['sentences_len_te']
    slots_dict = data['slots_dict']
    intents_dict = data['intents_dict']
    one_hot_intents_te = data['encoded_intents_te']
    one_hot_slots_te = data['encoded_slots_te']

    embedding = data['embedding']

#     flags.set_data_flags(data)

    # percentage split
    print('------------------start subsampling-------------------')
    subsampling_runs = 5
    fold = 1
    intent_scores = []
    slot_scores = []
    
    # Randomly sample 90% of training data
    for train_index, val_index in StratifiedShuffleSplit(n_splits=5, train_size=0.9).split(x_tr,y_intents_tr):
        print('RUN %d\n' % fold)
    
        print("Train index: " + str(len(train_index)))
        print(train_index)
        print("Val index: " + str(len(val_index)))
        print(val_index)

        # Split the data according to train_index
        x_train = x_tr[train_index]
        print(len(x_train))
        y_intents_train = y_intents_tr[train_index]
        y_slots_train = y_slots_tr[train_index]
        sentences_length_train = sentences_length_tr[train_index]
        one_hot_intents_train = one_hot_intents_tr[train_index]
        one_hot_slots_train = one_hot_slots_tr[train_index]

        x_val = x_tr[val_index]
        y_intents_val = y_intents_tr[val_index]
        y_slots_val = y_slots_tr[val_index]
        sentences_length_val = sentences_length_tr[val_index]
        one_hot_intents_val = one_hot_intents_tr[val_index]
        one_hot_slots_val = one_hot_slots_tr[val_index]

        # Randomly sample 10% of test data
        _, test_index = StratifiedShuffleSplit(n_splits=2, test_size=0.1).split(x_te, y_intents_te)
        # Split the data according to test_index
        test_index = test_index[0]
        print("Test index: " + str(len(test_index)))
        print(test_index)
        x_test = x_te[test_index]
        y_intents_test = y_intents_te[test_index]
        y_slots_test = y_slots_te[test_index]
        sentences_length_test = sentences_length_te[test_index]
        one_hot_intents_test = one_hot_intents_te[test_index]
        one_hot_slots_test = one_hot_slots_te[test_index]

        train_data = dict()
        train_data['x_tr'] = x_train
        train_data['y_intents_tr'] = y_intents_train
        train_data['y_slots_tr'] = y_slots_train
        train_data['sentences_len_tr'] = sentences_length_train
        train_data['one_hot_intents_tr'] = one_hot_intents_train
        train_data['one_hot_slots_tr'] = one_hot_slots_train

        val_data = dict()
        val_data['x_val'] = x_val
        val_data['y_intents_val'] = y_intents_val
        val_data['y_slots_val'] = y_slots_val
        val_data['sentences_len_val'] = sentences_length_val
        val_data['one_hot_intents_val'] = one_hot_intents_val
        val_data['one_hot_slots_val'] = one_hot_slots_val
        val_data['slots_dict'] = slots_dict
        val_data['intents_dict'] = intents_dict

        test_data = dict()
        test_data['x_te'] = x_test
        test_data['y_intents_te'] = y_intents_test
        test_data['y_slots_te'] = y_slots_test
        test_data['sentences_len_te'] = sentences_length_test
        test_data['one_hot_intents_te'] = one_hot_intents_test
        test_data['one_hot_slots_te'] = one_hot_slots_test
        test_data['slots_dict'] = slots_dict
        test_data['intents_dict'] = intents_dict
        
        vocab_size, word_emb_size = data['embedding'].shape
        _, max_sentence_length = data['x_tr'].shape
        intents_number = len(data['intents_dict'])
        slots_number = len(data['slots_dict'])
        
        del_some_flags(FLAGS)
        tf.app.flags.DEFINE_string('scenario_num', scenario_nums[i], 'Scenario number')
        tf.app.flags.DEFINE_integer('vocab_size', vocab_size, 'vocab size of word vectors')
        tf.app.flags.DEFINE_integer('max_sentence_length', max_sentence_length, 'max number of words in one sentence')
        tf.app.flags.DEFINE_integer('intents_nr', intents_number, 'intents_number')  #
        tf.app.flags.DEFINE_integer('slots_nr', slots_number, 'slots_number')  #
        tf.app.flags.DEFINE_integer('word_emb_size', word_emb_size, 'embedding size of word vectors')

        # Train and test on split
        f_intent, f_slot = percentage_split.train_and_test(train_data, val_data, test_data, embedding, FLAGS, fold)

        fold += 1

        # For each iteration, add score to list
        intent_scores.append(f_intent)
        slot_scores.append(f_slot)

    # Compute standard deviation
    std_intent = np.std(intent_scores)
    std_slot = np.std(slot_scores)
    print('Std intent F1 score %lf' % std_intent)
    print('Std slot F1 score %lf' % std_slot)

    run_results[scenario_nums[i]] = dict()
    run_results[scenario_nums[i]]['intent_scores'] = intent_scores
    run_results[scenario_nums[i]]['slot_scores'] = slot_scores
    run_results[scenario_nums[i]]['std_intent'] = std_intent
    run_results[scenario_nums[i]]['std_slots'] = std_slot

percentage_split.write_csv(run_results)

#     flags.del_all_flags(FLAGS)


Reading training data from ../data-capsnets/scenario0/train.txt
Reading test data from ../data-capsnets/scenario0/train.txt
Scenario num 0_perc
------------------read datasets begin-------------------
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabulary!
Word RockTV not in W2V vocabu

------------------epoch :  13  Loss:  0.08511542 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 1.000000
F score 1.000000
Slot filling
F1 score: 0.980545
Accuracy: 0.998291
Current F score mean 0.9902723735408561
Best F score mean 0.9902723735408561
------------------epoch :  14  Loss:  0.091986775 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 1.000000
F score 1.000000
Slot filling
F1 score: 0.968992
Accuracy: 0.997721
Current F score mean 0.9844961240310077
Best F score mean 0.9902723735408561
------------------epoch :  15  Loss:  0.063385546 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 1.000000
F score 1.000000
Slot filling
F1 score: 0.980545
Accuracy: 0.998291
Current F score mean 0.9902723735408561
Best F score mean 0.9902723735408561
------------------epoch :  16  Loss:  0.0646261 ----------------------
           V

------------------epoch :  8  Loss:  0.12636968 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.991453
F score 0.991453
Slot filling
F1 score: 1.000000
Accuracy: 1.000000
Current F score mean 0.9957264957264957
Best F score mean 0.9957264957264957
------------------epoch :  9  Loss:  0.12308588 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.965812
F score 0.965812
Slot filling
F1 score: 1.000000
Accuracy: 1.000000
Current F score mean 0.982905982905983
Best F score mean 0.9957264957264957
------------------epoch :  10  Loss:  0.10573711 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.965812
F score 0.965812
Slot filling
F1 score: 1.000000
Accuracy: 1.000000
Current F score mean 0.982905982905983
Best F score mean 0.9957264957264957
------------------epoch :  11  Loss:  0.09332384 ----------------------
           VALIDA

           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.965812
F score 0.965812
Slot filling
F1 score: 1.000000
Accuracy: 0.998291
Current F score mean 0.982905982905983
Best F score mean 0.982905982905983
------------------epoch :  4  Loss:  0.36992908 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 1.000000
F score 1.000000
Slot filling
F1 score: 1.000000
Accuracy: 1.000000
Current F score mean 1.0
Best F score mean 1.0
------------------epoch :  5  Loss:  0.17682537 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 1.000000
F score 1.000000
Slot filling
F1 score: 1.000000
Accuracy: 1.000000
Current F score mean 1.0
Best F score mean 1.0
------------------epoch :  6  Loss:  0.18915613 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 1.000000
F score 1.000000
Slot filling
F1 score: 1.000000
Accuracy: 1.0

           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.076923
F score 0.076923
Slot filling
F1 score: 0.024662
Accuracy: 0.111111
------------------epoch :  0  Loss:  12.276939 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.666667
F score 0.666667
Slot filling
F1 score: 0.605578
Accuracy: 0.952137
Current F score mean 0.6361221779548474
Best F score mean 0.6361221779548474
------------------epoch :  1  Loss:  2.8821416 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.871795
F score 0.871795
Slot filling
F1 score: 0.817121
Accuracy: 0.986325
Current F score mean 0.8444577471814825
Best F score mean 0.8444577471814825
------------------epoch :  2  Loss:  1.6158261 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.974359
F score 0.974359
Slot filling
F1 score: 0.945736
Accuracy: 0.995442
Current F sc

...

------------------epoch :  3  Loss:  2.8114927 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.949153
F score 0.949153
Slot filling
F1 score: 0.980198
Accuracy: 0.997175
Current F score mean 0.9646752810874308
Best F score mean 0.9646752810874308
------------------epoch :  4  Loss:  2.8948073 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.949153
F score 0.949153
Slot filling
F1 score: 1.000000
Accuracy: 1.000000
Current F score mean 0.9745762711864407
Best F score mean 0.9745762711864407
------------------epoch :  5  Loss:  0.9689276 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.966102
F score 0.966102
Slot filling
F1 score: 1.000000
Accuracy: 1.000000
Current F score mean 0.9830508474576272
Best F score mean 0.9830508474576272
------------------epoch :  6  Loss:  0.27942708 ----------------------
           VALIDATIO

Test index: 350
[274 265 333  70 181 270 272 113 381 166 354 364 230  92  23   7 244 116
  76 102 119 109  47 346 331 316 207  25 206  35 375  58 307  79 196 190
 297 246  31 359 163 322 120 279 152 143 357 232 343  26 223 275 344 112
 250 194 383  98 330 380 202 236 188 110 289 371 121 386 260 222  51 117
 376 320  99  37   2 133 111  48 349 323 299  16 309 241 303  80 318  89
  78 123  65 211   1 129 239  18 382  82 198 273   3 136 245 203  71 101
 377 348  20  96 287  95  97 137  84  74 173 332 149 368 262 365 122  86
  52 327 215 252 286  28 170 372 217 355 156 227 335  36 189 308  64  32
  72 255   5 290 171 305  75  88  41 369 253 345 210 243 131 118 124 180
 125 313  43 296  50 205  30  61  40 162 266 235 328 281 105  33 258 362
 351 363 157 306  81   0 304 204 193 141 219 350 242 115  45 338 208 294
 128  63 240 341 237 280  53  38 174 155 138 256 292  77 224 209 271 191
  59 165  24 254 185 107 144  68  10  69 229 201  90 370 221  94 216  13
 267 160 340 199 358  66  14 187 17

RUN 4

Train index: 526
[171 458 100 439 407  22 140  68  23 146 143 361 466   7 248 534 333 318
 198 543 131 281 297 442 549 493 149 206 425 342 254 290 556 127 226 494
  19 457 502 284 180 293  80 305 324 190 554 515 332 227 340 539  25 315
  14 322 174  10 242 219 420 270 234 496 295 225 476 194 488 232 125  60
 251  55 530  64 215 359 193 104 573  76  71 463 462 263 353  11 431 325
 436 569 106 434 471 405 341 336 478 117 448 159  47 370  84 389 306 338
 136 308 479 540 575  61 291  97  99 371 529 229 381 499 244 346 177 510
 422 236 380 419 497 139 445 532 103 570 364 360 375 296 246 287 507 520
 506 455  21  48 355   4  33 273 563 372  44  30  53 536 576  92 412 474
 141 288 523 301 132  46 178 311 358 433 144 167 498 135 577 382 160 304
 459  63 557  70 584  72 542 191 183  20 264 120 417  90 329  13 406 289
 413 492 314 114 321 579 237  26 432 582 211 391 477 172 256 521 175 456
 276 200 247 282 511 142 396 107  40 157 186  62  31 222 504 216  49  42
 238 369 518 580 272 354  7

------------------epoch :  14  Loss:  0.08409974 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.983051
F score 0.991453
Slot filling
F1 score: 0.989899
Accuracy: 0.998588
Current F score mean 0.9906759906759908
Best F score mean 0.9906759906759908
------------------epoch :  15  Loss:  0.0674226 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.983051
F score 0.991453
Slot filling
F1 score: 0.989899
Accuracy: 0.998588
Current F score mean 0.9906759906759908
Best F score mean 0.9906759906759908
------------------epoch :  16  Loss:  0.08219652 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.983051
F score 0.991453
Slot filling
F1 score: 0.989899
Accuracy: 0.998588
Current F score mean 0.9906759906759908
Best F score mean 0.9906759906759908
------------------epoch :  17  Loss:  0.073965386 ----------------------
           VA

------------------epoch :  3  Loss:  0.7763968 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.932203
F score 0.932203
Slot filling
F1 score: 0.974093
Accuracy: 0.995763
Current F score mean 0.9531483270396066
Best F score mean 0.9622175141242937
------------------epoch :  4  Loss:  1.26511 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.949153
F score 0.949153
Slot filling
F1 score: 0.979381
Accuracy: 0.997175
Current F score mean 0.9642669928359252
Best F score mean 0.9642669928359252
------------------epoch :  5  Loss:  0.36869437 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.983051
F score 0.991453
Slot filling
F1 score: 0.994819
Accuracy: 0.998588
Current F score mean 0.9931358221513662
Best F score mean 0.9931358221513662
------------------epoch :  6  Loss:  0.734512 ----------------------
           VALIDATION S

In [21]:
def write_csv(run_results):
    with open('percentage_split_results.csv', 'w') as f:
        intent_title_line = ['Intent detection']
        header_line = ['Scenario', 'Run 1', 'Run 2', 'Run 3', 'Run 4', 'Run 5', 'Std']
        writer = csv.writer(f)
        writer.writerow(intent_title_line)
        writer.writerow(header_line)
        for k, v in run_results.items():
            scenario_num = [k]
            runs_scores = v['intent_scores']
            std = ['{:.3f}'.format(v['std_intent'])]
            l = scenario_num + runs_scores + std
            writer.writerow(l)

        intent_title_line = ['Slot filling']
        header_line = ['Scenario', 'Run 1', 'Run 2', 'Run 3', 'Run 4', 'Run 5', 'Std']
        writer = csv.writer(f)
        writer.writerow(intent_title_line)
        writer.writerow(header_line)
        for k, v in run_results.items():
            scenario_num = [k]
            runs_scores = v['slot_scores']
            std = ['{:.3f}'.format(v['std_slots'])]
            l = scenario_num + runs_scores + std
            writer.writerow(l)